# Data Cleaning

This script is for tranforming initial raw data to input vectors. The output files should be of 4 types: sem_items, sem_relations, sem_attributes, sem_data.

- sem_items, sem_relations, and sem_attributes consist of all names of items, relations and attributes. These 3 files will not enter model training directly, however, they will supplement in other functions like analysis plotting, so we still recommend processing and storing them in full.
- sem_data consists of all one-hot encoded vectors as input for training model.

Check sample_input in data folder for reference of output.

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!ls "/content/drive/My Drive/1016 Cognition Project/"
path = "/content/drive/My Drive/1016 Cognition Project/"

# Subsample the original dataset

In [ ]:
#read data
object_data = pd.read_excel(path+"Concept_similarity_matrix.xlsx")
object_property = pd.read_excel(path+"All_concepts_and_features.xlsx")

In [ ]:
object_data.head()

,Unnamed: 0,Unnamed: 1,安全带,安全帽,安慰奶嘴,鹌鹑,鹌鹑蛋,按键,按摩器,按钮,八宝粥,八哥,巴士,芭蕾舞裙,霸王龙,吧台,白菜,白饭,白果,白桦树,白酒,白萝卜,白皮松,白砂糖,白水,白糖,白杨,白蚁,白银,白粥,百合,百科全书,百事可乐,百香果,柏树,柏油路,摆钟,扳手,斑马,板栗,...,蜘蛛,直立式钢琴,直升机,止血剂,纸,纸巾,指挥棒,指甲,指甲锉,指甲油,指南针,指纹,制服,痣,中山装,中提琴,钟,粥,猪,猪笼草,猪肉,竹节虫,竹子,烛台,煮蛋器,助听器,注射器,蛀虫,砖头,桌布,啄木鸟,子弹,自动步枪,自动扶梯,自动售货机,自行车,字典,钻石,嘴唇,座垫
0,NaN,NaN,Safety belt,Safety hat,pacifier,quail,Quail egg,Key,Massager,Button,Mixed congee,Myna,Bus,ballet skirt,Tyrannosaurus Rex,Bar counter,Chinese cabbage,Plain white rice,Ginkgo biloba,silver birch,Liquor and Spirits,ternip,white-barked pine,White granulated sugar,plain water,White sugar,poplar,termite,silver,Rice Congee,Lily,encyclopedia,Pepsi Cola,Passion fruit,Cypress,Tar Road,Pendulum,Wrench,zebra,Chinese chestnut,...,spider,Upright piano,helicopter,Hemostatic Agents,paper,tissue,Baton,nail,Nail file,Nail Polish,Compass,fingerprint,uniform,nevus,Chinese tunic suit,Viola,clock,Porridge,pig,Nepenthes,pork,stick insect,Bamboo,Candlestick,Egg Cooker,Deaf-aid,Syringe,moth,brick,tablecloth,Woodpecker,Bullet,Automatic rifle,Escalator,vending machine,Bicycle,Dictionaries,Diamonds,Lip,Cushion
1,安全带,Safety belt,1,0.414539,-0.0125689,-0.0957668,-0.0483416,-0.00209044,0.00247259,0.0130681,-0.0458548,-0.0875633,-0.0174352,-0.0333406,-0.0954928,0.0286141,-0.109101,-0.0683503,-0.107826,-0.0881527,-0.0515634,-0.110055,-0.0893827,-0.022952,-0.0453074,-0.0640332,-0.0970787,-0.092205,-0.0320813,-0.0491984,-0.115517,-0.00017585,-0.0106182,-0.0958466,-0.0949719,0.0140069,-0.015865,-0.014933,-0.0900823,-0.114536,...,-0.0714185,-0.00431414,-0.017217,-0.0184519,-0.0197497,-0.00748598,0.0554013,0.342369,-0.0300324,0.322542,0.00103401,-0.00235393,-0.02115,-0.019971,-0.0175151,-0.00960751,0.00784836,-0.0296317,-0.11552,-0.0956361,-0.0236798,-0.0642364,-0.0841961,-0.0262817,-0.0543253,-0.00105258,-0.00841526,-0.0732531,-0.075671,0.321923,0.165374,-0.0276467,0.293427,-0.00390994,0.000605374,-0.0160302,4.12382e-05,-0.0281335,0.306865,0.349275
2,安全帽,Safety hat,0.414539,1,-0.0187715,-0.117925,-0.0220425,-0.00283529,-0.0348581,0.0362228,-0.0781839,-0.115681,-0.0340659,-0.0348317,-0.122962,0.0715859,-0.117735,-0.108662,-0.0554468,-0.113601,-0.0675215,-0.126217,-0.113442,-0.0521456,-0.0764681,-0.0890257,-0.116312,-0.108439,0.0739891,-0.0692351,-0.116803,-0.0239901,-0.0442125,-0.0688177,-0.112971,0.0549857,0.028187,0.108984,-0.122533,0.029822,...,-0.0942894,-0.0227235,-0.0394498,-0.0508225,-0.0175678,-0.0348959,-0.0268329,0.409498,0.119719,0.318777,-0.0286148,-0.0443208,-0.0166787,-0.0516413,-0.015394,-0.0288057,-0.03153,-0.0527438,-0.142505,-0.141565,-0.0512213,-0.113184,-0.143898,0.110731,0.0451497,-0.0211359,-0.0270686,-0.125619,0.0473173,0.30762,0.209814,0.0587157,0.351328,0.0725466,-0.0475625,-0.0400154,0.00619625,0.0620956,0.293337,0.335119
3,安慰奶嘴,pacifier,-0.0125689,-0.0187715,1,-0.105157,-0.0274761,0.100864,0.0923346,0.0540921,0.000155649,-0.093809,0.0499033,0.0174043,-0.0958972,0.0666528,-0.104779,-0.0205999,-0.0907499,-0.0914092,-0.00426837,-0.115078,-0.0920859,0.082621,-0.00270464,-0.00712339,-0.0979111,-0.0996138,0.0645259,0.00506848,-0.133738,0.0772587,0.078913,-0.116647,-0.0950318,0.0411831,0.0573223,0.113808,-0.102963,-0.111234,...,-0.0758094,0.0459779,0.0439349,0.0774315,0.0312468,0.0806492,0.0846025,-0.00877074,0.0974599,-0.00712091,0.111412,0.142214,0.0130985,0.104188,0.025351,0.0552852,0.0719511,0.0769303,-0.109292,-0.142482,0.0730405,-0.0958587,-0.109122,0.0561859,0.00646419,0.0636279,0.141721,-0.112623,-0.00352667,0.00865549,-0.147928,0.0509535,0.0125637,0.048956,0.10863,0.0845791,0.0733384,0.0289418,-0.0139314,0.0142867
4,鹌鹑,quail,-0.0957668,-0.117925,-0.105157,1,0.0421095,-0.149978,-0.103923,-0.0605614,0.0678059,0.792691,-0.0955625,-0.127287,0.369394,-0.118978,0.00709994,0.0695488,0.0381056,-0.0446143,-0.034203,0.000466475,-0.0458438,-0.0145633,-0.0214586,0.0572243,-0.0523434,0.410488,-0.13688,0.0508003,0.001112,-0.08181,-0.0954412,0.0090552,-0.0512503,-0.103747,-0.103511,-0.153287,0.375457,0.0309249,...,0.258262,-0.0637059,0.0393613,-

In [ ]:
all_objects = object_data.iloc[1:,1]
all_objects = np.array(all_objects)

In [ ]:
#randomly choose 700 objects 
object_sample = np.random.choice(all_objects, size = 700, replace = False)

In [ ]:
object_property.head()

,概念,concept,特征,feature,填写该特征被试数-number of participants mentioning this feature,填写该概念被试数-number of participants rating this feature,按30人计算有多少人填写了该特征-normalized numer of participants mentioning this feature
0,安全带,Safety belt,可以-保护,can-Protect,28,31,27.096774
1,安全带,Safety belt,是-带子,is-tape,23,31,22.258065
2,安全带,Safety belt,其他-车,other-Cars,18,31,17.419355
3,安全带,Safety belt,是-安保的,is-security,17,31,16.451613
4,安全带,Safety belt,可以-保护人,can-Protect People,16,31,15.483871


In [ ]:
#subsample object-property data
object_property_sub = object_property[object_property['concept'].isin(object_sample)]

In [ ]:
#save to a new data
object_property_sub.to_csv(path+'concept_feature_sub.csv')

# Data cleaning, one-hot encoding

In [ ]:
#load data
data = pd.read_csv(path+"0509_1.csv")

In [ ]:
data

,__,concept,__.1,feature,________-number of participants mentioning this feature,________-number of participants rating this feature,_30_____________-normalized numer of participants mentioning this feature,COUNTIF feature,COUNTIF item,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,___,Tyrannosaurus Rex,__-_,need-Water,13,28,13.928571,390,16,NaN,NaN,NaN,NaN
1,___,silver birch,__-_,need-Water,17,30,17.000000,390,19,NaN,NaN,NaN,NaN
2,___,white-barked pine,__-_,need-Water,16,30,16.000000,390,16,NaN,NaN,NaN,NaN
3,__,Cypress,__-_,need-Water,18,28,19.285714,390,16,NaN,NaN,NaN,NaN
4,__,zebra,__-_,need-Water,14,28,15.000000,390,25,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1494,___,Broccoli,__-_,can-Stir-fry,10,30,10.000000,50,16,NaN,NaN,NaN,NaN
1495,__,sausage,__-_,can-Stir-fry,9,31,8.709677,50,18,NaN,NaN,NaN,NaN
1496,___,A variety of Chinese cabbage,__-_,can-Stir-fry,8,30,8.000000,50,17,NaN,NaN,NaN,NaN
1497,__,onion,__-_,can-Stir-fry,12,30,12.000000,50,17,NaN,NaN,NaN,NaN


In [ ]:
#split feature into relation and property
data['feature'] = data['feature'].str.split('-')

In [ ]:
#reset index 
data = data.reset_index(drop=True)

In [ ]:
#separate feature
data['relation'] = [data['feature'][i][0] for i in range(len(data))]
data['attribute'] = [data['feature'][i][1] for i in range(len(data))]

In [ ]:
data['attribute'] = data['attribute'].str.lower()

## Concept

In [ ]:
#sort data according to concept
data = data.sort_values(by=['concept'])
#reset index 
data = data.reset_index(drop=True)

In [ ]:
#one-hot encoding
from sklearn.preprocessing import LabelBinarizer
#from sklearn.preprocessing import OneHotEncoder
data['concept_encoded'] = LabelBinarizer().fit_transform(data.concept).tolist()

In [ ]:
#save concept into a txt file, comment it when done

all_concepts = data.concept.unique().tolist()
with open(path+'sem_items_0509_1.txt', 'w+') as f:
 
    # write elements of list
    for items in all_concepts:
        f.write('%s\n' %items)
      
    print("File written successfully")
# close the file
f.close()


File written successfully


## Relation

In [ ]:
data.relation.unique()

array(['is', 'need', 'has', 'can'], dtype=object)

In [ ]:
#clean relation
data.loc[(data['relation'] == 'laike') | (data['relation'] == 'likes '), 'relation'] = 'like'

In [ ]:
#sort data according to relation
data = data.sort_values(by=['concept','relation'])

In [ ]:
#one-hot encoding
data['relation_encoded'] = LabelBinarizer().fit_transform(data.relation).tolist()

In [ ]:
#save relation into a txt file, comment it when done

all_relations = data.relation.unique().tolist()
with open(path+'sem_relations_0509_1.txt', 'w+') as f:
 
    # write elements of list
    for items in all_relations:
        f.write('%s\n' %items)
      
    print("File written successfully")
# close the file
f.close()


File written successfully


## Attribute

In [ ]:
#trim spaces
data['attribute'] = data['attribute'].str.strip()

In [ ]:
#sort data according to attribute
data = data.sort_values(by=['attribute'])

In [ ]:
#one-hot encoding
data['attribute_encoded'] = LabelBinarizer().fit_transform(data.attribute).tolist()

In [ ]:
#save relation into a txt file, comment it when done

all_attributes = data.attribute.unique().tolist()
with open(path+'sem_attributes_0509_1.txt', 'w+') as f:
 
    # write elements of list
    for items in all_attributes:
        f.write('%s\n' %items)
      
    print("File written successfully")
# close the file
f.close()


File written successfully


In [ ]:
#concatenate encoded columns
data = data.sort_values(by=['concept','relation'])
data = data.reset_index(drop=True)
data['encoded_concat'] = [data['concept_encoded'][i]+data['relation_encoded'][i]+data['attribute_encoded'][i] for i in range(len(data))]

In [ ]:
data.to_csv(path+'encoded_data.csv')

# Get encoded sequences

In [ ]:
data = pd.read_csv(path+'encoded_data.csv')

In [ ]:
#convert str of list to list
from ast import literal_eval
data.concept_encoded = data.concept_encoded.apply(literal_eval)
data.relation_encoded = data.relation_encoded.apply(literal_eval)
data.attribute_encoded = data.attribute_encoded.apply(literal_eval)

In [ ]:
type(data['attribute_encoded'][0])

list

In [ ]:
#convert encoded attribute to np array
data.attribute_encoded = data.attribute_encoded.apply(np.array)

In [ ]:
type(data['attribute_encoded'][0])

numpy.ndarray

In [ ]:
g = data.groupby(['concept', 'relation'])
attr_concat = pd.concat([g.attribute_encoded.apply(np.sum)], axis=1)

In [ ]:
len(attr_concat['attribute_encoded'])

497

In [ ]:
concept_relation = [data['concept_encoded'][i]+data['relation_encoded'][i] for i in range(len(data))]
len(concept_relation)

1499

In [ ]:
unique_concept_relation = []
for i in concept_relation:
    if i not in unique_concept_relation:
        unique_concept_relation.append(i)

In [ ]:
len(unique_concept_relation)

497

In [ ]:
attr_concat_l = attr_concat.attribute_encoded.apply(np.ndarray.tolist)

In [ ]:
obj_rel_attr_list = [unique_concept_relation[i]+attr_concat_l[i] for i in range(len(unique_concept_relation))]

In [ ]:
#convert list of int to string
obj_rel_attr_str_list = []
for l in obj_rel_attr_list:
    string_l = [str(n) for n in l]
    str_of_int = " ".join(string_l)
    obj_rel_attr_str_list.append(str_of_int)

In [ ]:
#save data into a txt file, comment it when done

with open(path+'sem_data_0429_2.txt', 'w+') as f:
 
    # write elements of list
    for items in obj_rel_attr_str_list:
        f.write('%s\n' %items)
      
    print("File written successfully")
# close the file
f.close()


File written successfully


In [ ]:
D = np.loadtxt(path+'sem_data.txt')